In [ ]:
!nvidia-smi

Tue Nov 22 04:48:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Research/Chatbot_GRU

/content/drive/MyDrive/Research/Chatbot_GRU


In [ ]:
!ls

Chatbot_GRU.ipynb	     formatted_dialogs_test.txt
Chatbot_Transformer.ipynb    formatted_dialogs.txt
data			     formatted_hindi_test.txt
dialogues_test.txt	     formatted_hindi_train1.txt
dialogues_train.txt	     formatted_hindi_train3.txt
en-indic.zip		     formatted_hindi_train.txt
formatted_dialogs1.txt	     Movie_dialogs.json
formatted_dialogs_test1.txt  Preprocessing.ipynb


In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (386/386), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 683 (delta 265), reused 266 (delta 190), pack-reused 297
Receiving objects: 100% (683/683), 2.65 MiB | 9.75 MiB/s, done.
Resolving deltas: 100% (392/392), done.
/content/drive/MyDrive/Research/Chatbot_GRU/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1325 (delta 91), reused 82 (delta 82), pack-reused 1218
Receiving objects: 100% (1325/1325), 9.55 MiB | 9.41 MiB/s, done.
Resolving deltas: 100% (701/701), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving obje

In [ ]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
!pip install mosestokenizer subword-nmt
# # Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 4.8 MB/s 
     |████████████████████████████████| 118 kB 69.9 MB/s 
     |████████████████████████████████| 125 kB 66.6 MB/s 
     |████████████████████████████████| 40 kB 5.8 MB/s 
     |████████████████████████████████| 2.8 MB 53.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=79ada199e89cfde51254fd77f19a8869a06ccc2f9841f19082afad263207f6c8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49188 sha256=b77e2cde5413cbf8020ac88e294b1de2237ef23041fcff1e4338b1f8679eeaf8
  Stored in directory: /root/.cache/pip/wheels/b0/35

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

Error No module named 'triton'
Error No module named 'triton'


In [ ]:
# download the indictrans model


# downloading the indic-en model
# !wget https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
# !unzip indic-en.zip

# downloading the en-indic model
# !wget https://storage.googleapis.com/samanantar-public/V0.3/models/en-indic.zip
!unzip en-indic.zip

# # # downloading the indic-indic model
# # !wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
# # !unzip m2m.zip

%cd indicTrans

Archive:  en-indic.zip
   creating: en-indic/
   creating: en-indic/vocab/
  inflating: en-indic/vocab/bpe_codes.32k.SRC  
  inflating: en-indic/vocab/vocab.SRC  
  inflating: en-indic/vocab/vocab.TGT  
  inflating: en-indic/vocab/bpe_codes.32k.TGT  
   creating: en-indic/final_bin/
  inflating: en-indic/final_bin/preprocess.log  
  inflating: en-indic/final_bin/dict.TGT.txt  
  inflating: en-indic/final_bin/test.SRC-TGT.SRC.idx  
  inflating: en-indic/final_bin/test.SRC-TGT.TGT.idx  
  inflating: en-indic/final_bin/train.SRC-TGT.SRC.idx  
  inflating: en-indic/final_bin/dict.SRC.txt  
  inflating: en-indic/final_bin/valid.SRC-TGT.TGT.idx  
  inflating: en-indic/final_bin/test.SRC-TGT.TGT.bin  
  inflating: en-indic/final_bin/valid.SRC-TGT.TGT.bin  
  inflating: en-indic/final_bin/train.SRC-TGT.TGT.idx  
  inflating: en-indic/final_bin/train.SRC-TGT.TGT.bin  
  inflating: en-indic/final_bin/valid.SRC-TGT.SRC.idx  
  inflating: en-indic/final_bin/train.SRC-TGT.SRC.bin  
  inflating: en-

In [ ]:
from inference.engine import Model

indic2en_model = Model(expdir='../en-indic')

Initializing vocab and bpe
Initializing model for translation


In [ ]:
%cd ..

/content/drive/MyDrive/Research/Chatbot_GRU


In [ ]:
%pwd

'/content/drive/MyDrive/Research/Chatbot_GRU'

In [ ]:
import csv
from tqdm import tqdm

In [ ]:
base='/content/drive/MyDrive/Research/Chatbot_GRU/'
formatted_dialogs_file=os.path.join(base,'formatted_dialogs1.txt')
formatted_hindi_train=os.path.join(base,'formatted_hindi_train3.txt')

formatted_dialogs_file_test=os.path.join(base,'formatted_dialogs_test1.txt')
formatted_hindi_test=os.path.join(base,'formatted_hindi_test.txt')

In [ ]:
def loadConversation(file):
  lines=[]
  with open(file,'r') as f:
    for line in f:
      lines.append(line.strip().split('\n'))
  return lines

In [ ]:
lines=loadConversation(formatted_dialogs_file)
print(len(lines))

76053


In [ ]:
print('Writing newly formatted hindi file...')
with open(formatted_hindi_train,'w',encoding='utf-8') as op:
  writer=csv.writer(op,delimiter='\t',lineterminator='\n')
  for line in tqdm(lines[40000:60000]):
    pair=line[0].split('\t')
    pair=indic2en_model.batch_translate(pair, 'en', 'hi')
    writer.writerow(pair)

Writing newly formatted hindi file...


Streaming output truncated to the last 5000 lines.
100%|██████████| 2/2 [00:00<00:00, 2178.86it/s]

100%|██████████| 2/2 [00:00<00:00, 3032.76it/s]

100%|██████████| 2/2 [00:00<00:00, 3416.95it/s]

100%|██████████| 2/2 [00:00<00:00, 652.25it/s]


100%|██████████| 2/2 [00:00<00:00, 542.71it/s]


Streaming output truncated to the last 5000 lines.
100%|██████████| 2/2 [00:00<00:00, 3347.41it/s]

100%|██████████| 2/2 [00:00<00:00, 3338.09it/s]

100%|██████████| 2/2 [00:00<00:00, 3583.34it/s]

100%|██████████| 2/2 [00:00<00:00, 3492.34it/s]

100%|██████████| 20000/20000 [2:09:05<00:00,  2.58it/s]


In [ ]:
a=[1,2,3,4,5]
a[:3]

[1, 2, 3]